In [1]:
import pandas as pd

reviews_data = pd.read_excel("C:/Users/Getzbie Alfredo Tpoy/Documents/4_SMT_UNUD/PPDM/Final Project PPDM/reviews.xlsx")

###### 1) Feature Extraction untuk memperoleh nilai-nilai bobot TF atau TFIDF dari data ulasan. Tahapan feature extraction: tokenization, lower case converting, stop word removal, stemming, dan pembobotan (TF atau TFIDF). Untuk metode Multinomial Naive Bayes pembobotan kata menggunakan formula TF, sedangkan untuk metode yang lain menggunakan TFIDF. Total jumlah fitur dari setiap data text tergantungdari panjang/ jumlah vocabulary setelah preprocessing (jumlah term indeks) dari koleksi ulasan yang digunakan.

###### 2) Feature Selection dengan menggunakan formula Chi-Square (lihat penjelasan di URL terkait), gunakan beberapa variasi jumlah fitur yang dipertahankan (10% atau 20% atau 30%, dsb).

In [2]:
reviews_data.head()

No                                            Reviews  Label
0   1  kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...      1
1   2  Jahitannya sih rapi,cuman ada benang yang ikut...      0
2   3  Sesuai harga. Agak tipis tapi masih oke kok. W...      0
3   4  Wah gila sihhh sebagus itu, se worth it, se  l...      1
4   5  Kain nya bagus halus  \nTapi kok di bukak koto...      0

### Lower case Converting

In [3]:
reviews_data["Reviews"] = reviews_data["Reviews"].str.lower()

In [4]:
print("Lower case Converting :\n")
print(reviews_data['Reviews'].head(10))
print("\n\n")

Lower case Converting :

0    kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...
1    jahitannya sih rapi,cuman ada benang yang ikut...
2    sesuai harga. agak tipis tapi masih oke kok. w...
3    wah gila sihhh sebagus itu, se worth it, se  l...
4    kain nya bagus halus  \ntapi kok di bukak koto...
5    baguss dehh dengan harga segitu kainnya juga g...
6    harga mahal bahan tipis banget. pengiriman lam...
7    puas bangett, awalnya takut jelek soalnya buat...
8    agak kecewa sihh karna pesan warna fuschia mal...
9    maaf tak kasi bintang 3 karena tdk sesuai gamb...
Name: Reviews, dtype: object





### Tokenizing, dan Cleansing

In [5]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist


def remove_Reviews_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_Reviews_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

reviews_data['Reviews'] = reviews_data['Reviews'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

reviews_data['Reviews_tokens'] = reviews_data['Reviews'].apply(word_tokenize_wrapper)

print('Hasil Tokenizing : \n') 
print(reviews_data['Reviews_tokens'].head())
print('\n\n')

Hasil Tokenizing : 

0    [kemeja, nya, bagusss, bgttttaaaa, mauuu, nngi...
1    [jahitannya, sih, rapicuman, ada, benang, yang...
2    [sesuai, harga, agak, tipis, tapi, masih, oke,...
3    [wah, gila, sihhh, sebagus, itu, se, worth, it...
4    [kain, nya, bagus, halus, tapi, kok, di, bukak...
Name: Reviews_tokens, dtype: object





### Stopwords removal

In [6]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "nya", "sih"])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

reviews_data['Reviews_Stopword'] = reviews_data['Reviews_tokens'].apply(stopwords_removal) 


print(reviews_data['Reviews_Stopword'].head())

0    [kemeja, bagusss, bgttttaaaa, mauuu, nngisssss...
1        [jahitannya, rapicuman, benang, jahit, jelek]
2    [sesuai, harga, tipis, oke, warnanya, abu, kal...
3    [gila, sihhh, sebagus, worth, it, lembut, baju...
4    [kain, bagus, halus, bukak, kotor, ya, warna, ...
Name: Reviews_Stopword, dtype: object


In [7]:
reviews_data.to_excel("preprocessing sementara.xlsx")

### Normalisasi

##### kata yang berupa kata gaul diubah menjadi kata normal, typo juga diubah menjadi kata normal. Setelah di normalisasi lakukaan Stopword removal ulang. Kata yang menjadi satu seperti 'nngisssssknpa' pada hasil review nomor 1 diubah menjadi  nangis','kenapa  agar tidak mengganggu hasil tokenisasi yang sudah dilakuakan diawal.

In [8]:

normalizad_word = pd.read_excel("normalisasi.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

reviews_data['reviews_normalized'] = reviews_data['Reviews_Stopword'].apply(normalized_term)

reviews_data['reviews_normalized'].head(10)

0    [kemeja, bagus, banget, mau, nangis kenapa, ti...
1       [jahitannya, rapi cuman, benang, jahit, jelek]
2    [sesuai, harga, tipis, oke, warnanya, abu, kal...
3    [gila, sih, sebagus, worth, it, lembut, bajuny...
4    [kain, bagus, halus, buka, kotor, ya, warna, p...
5    [bagus, deh, harga, segitu, kainnya, tidak, ti...
6    [harga, mahal, bahan, tipis, banget, pengirima...
7    [puas, banget, takut, jelek, foto, yearbook, p...
8    [kecewa, sih, karena, pesan, warna, fuschia, d...
9    [maaf, beri, bintang, tidak, sesuai, gambar, d...
Name: reviews_normalized, dtype: object

In [10]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in reviews_data['reviews_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

reviews_data['reviews_tokens_stemmed'] = reviews_data['reviews_normalized'].swifter.apply(get_stemmed_term)
print(reviews_data['reviews_tokens_stemmed'])

2575
------------------------
kemeja : kemeja
bagus : bagus
banget : banget
mau : mau
nangis kenapa : nang kenapa
tidak : tidak
dari : dari
dulu : dulu
beli : beli
ditoko : toko
kemejanya : kemeja
asli : asli
memang : memang
bahannya : bahan
juga : juga
adem : adem
gerahan : gerah
nerawang : nerawang
karena : karena
camera : camera
jelek : jelek
jadi : jadi
terlalu : terlalu
aslinya : asli
bohong : bohong
sumpah : sumpah
jahitannya : jahit
rapi cuman : rapi cuman
benang : benang
jahit : jahit
sesuai : sesuai
harga : harga
tipis : tipis
oke : oke
warnanya : warna
abu : abu
kalau : kalau
di foto : di foto
seperti : seperti
biru : biru
dikit : dikit
thanks : thanks
sellerr : sellerr
gila : gila
sih : sih
sebagus : bagus
worth : worth
it : it
lembut : lembut
bajunya : baju
kirain : kirain
saya : saya
kaos : kaos
dah : dah
dengan : dengan
segitu : segitu
thankyou : thankyou
ya : ya
next : next
order : order
lagi : lagi
kain : kain
halus : halus
buka : buka
kotor : kotor
warna : warna
putih 

Pandas Apply:   0%|          | 0/831 [00:00<?, ?it/s]

0      [kemeja, bagus, banget, mau, nang kenapa, tida...
1              [jahit, rapi cuman, benang, jahit, jelek]
2      [sesuai, harga, tipis, oke, warna, abu, kalau,...
3      [gila, sih, bagus, worth, it, lembut, baju, ki...
4      [kain, bagus, halus, buka, kotor, ya, warna, p...
                             ...                        
826    [terima, kasih, barang, sesuai, ukur, seesuai,...
827    [mantapp, realpicttt, bangttt, emas, cuman, pl...
828    [suka, banget, tas, tidak, seperti, tas, local...
829    [kualitas, produk, produk, original, harga, pr...
830    [barang, sudah, dengan, selamat, mantul, bange...
Name: reviews_tokens_stemmed, Length: 831, dtype: object


In [11]:
reviews_data.to_excel("reviews_Preprocessing.xlsx")